#### Setup

In [ ]:
from specific import *

### Load the dataframes for all experiments

In [ ]:
experiment_data = load_experiment_data(
    list(experiment_name_dict),
    which="data_split",
    ignore=("X_train", "y_train", "y_test"),
)

### Parse the dataframes to retrieve the variables used for each experiment

In [ ]:
contents = {}
for exp in sort_experiments(experiment_data):
    contents[exp] = shorten_features(
        repl_fill_names(sort_features(experiment_data[exp]["X_test"].columns))
    )

unique_vars = sort_features(
    [var for var in contents["all"] if not re.search("\s.?(\d+)M", var)]
)

### Build the matrix representing which variables are present with which lags for each of the experiments

In [ ]:
condensed = {}
for exp, exp_vars in contents.items():
    for var in unique_vars:
        # Find for which lags the current variable is present (if any).
        lags = [f"{get_lag(v)}M".replace("0M", "C") for v in exp_vars if var in v]
        if all(
            lag in lags for lag in ["C", "1M", "3M", "6M", "9M", "12M", "18M", "24M"]
        ):
            lags = "C & all A"
        else:
            lags = ", ".join(lags)
        condensed[(experiment_name_dict[exp], var)] = lags

In [ ]:
df = (
    pd.Series(condensed)
    .unstack()
    .reindex(index=[experiment_name_dict[c] for c in contents], columns=unique_vars)
)
df

In [ ]:
print(df.to_latex())